### **Naive Bayes Classifier**

**[Probability Overview]** - 머신러닝의 학습방법들
- Gradient descent based learning
- Probability theory based learning - 이번에 배울거
- Information theory based learning
- Distance similarity based learning

**Probability**
- 이산형 값
> ##### $P(X) = \frac{count(Event_X)}{count(ALL_{Event})}$

- 연속형 값
> $P(-\infty < x < \infty) = \int_{-\infty}^{\infty} f(x)dx = 1$
<br><br><br>

**Basic concepts of probability**
> $0 \le P(E) \le 1$ <br>
> $P(S) = \sum_{i=1}^N P(E_i) = 1$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;if all $E_i$ are independent
<br><br><br>

**Conditional probability**
> $P(A|B) = \frac{P(A \cap B)}{P(B)}$

<br><br>
**[Bayes's Theorem]**
- 경험에 의한 확률 업데이트
- 주어진 사전 확률에서 특정 사건 발생을 통해 사후 확률로 계속해서 업데이트(ex.개표방송)
- 빈도주의 vs 베이즈주의
- 객관적 확률은 존재하지 않는다!

**Probability updated**
- 특정 사건이 일어남에 따라 확률을 업데이트(ex. 퀸찾기 카드게임)

**Bayes's theorem**
> $P(A|B) = \frac{P(A \cap B)}{P(B)}$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$P(B|A) = \frac{P(A \cap B)}{P(A)}$ <br>
> $P(A \cap B) = P(B)P(A|B) = P(A)|P(B|A)$ <br><br>
> $P(A|B) = \frac{P(A \cap B)}{P(B)} = \frac{P(A)P(B|A)}{P(B)}$
- 다시말해서, 사건B가 일어났을 때의 사건A가 일어날 확률은, 사건B가 일어났을때, 사건A가 일어날 확률은 주어져있고, 사건A가 일어났을때 사건B가 일어날 확률을 거기에 곱해주면 된다

> $P(H|D) = \frac{P(H)P(D|H)}{P(D)}$ <br>
(H is Class &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; D is Data)
- P(H|D) : 사후확률, 어떤 데이터가 주어졌을 때의 가설이 발생할(발생하지않을) 확률 (H : 가설함수)
- P(D) : 전체확률, 데이터가 발생할 확률(Evidence)
- P(H) : 사전확률, 예) 1000개중 $y_1$= 700, $y_2$=300개면 P($H_1$)=700/1000
- 결국은 주어진 데이터(D) X를 통해 클래스(D) Y를 찾는 작업
- 나중에는 P(D)는 고정값이 되고, P(H)는 주어져서 결국 P(D|H), 우도를 찾는게 가장 중요해진다.

<br><br>
**[Naive Bayes Classifier]**<br><br>
예제로 풀어보기 - Viagra 스팸 필터기
- Viagra라는 단어의 유무를 통해 스팸 여부 확인
- Viagra 단어가 들어가면 무조건 스팸?
- 어느정도 확률로 스팸이라고 해야할까? <br><br>
![img](https://ifh.cc/g/3vSPxP.png)

- $P(spam | viagra) = \frac{P(spam)P(viagra|spam)}{P(viagra)}$

In [22]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [23]:
viagra_spam = {'viagra': [1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,1],
               'spam': [
                   1,0,0,0,0,0,1,0,1,0, 0,0,0,0,0,0,0,1,1,1
               ]}

df = pd.DataFrame(viagra_spam, columns=['viagra', 'spam'])
df.head()

,viagra,spam
0,1,1
1,0,0
2,0,0
3,0,0
4,0,0


In [24]:
np_data = df.values
np_data

array([[1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 1],
       [0, 1],
       [1, 1]], dtype=int64)

- P(viagra), P(spam), P(V $\cap$ S), P($N^c \cap S$) 구하기

In [19]:
p_viagra = sum(np_data[:, 0] == 1) / len(np_data)
p_spam = sum(np_data[:, 1] == 1) / len(np_data)
p_v_cap_s = sum((np_data[:, 0] == 1) & (np_data[:, 1] == 1)) / len(np_data)
p_n_v_cap_s = sum((np_data[:, 0] == 0) & (np_data[:, 1]== 1)) / len(np_data)

- P(Spam | Viagra)

In [20]:
p_spam * (p_v_cap_s / p_spam) / p_viagra

0.5

- P(Spam | ~Viagra)

In [21]:
p_spam * (p_n_v_cap_s / p_spam) / (1-p_viagra)

0.2142857142857143

<br><br>
**제대로된 스팸 필터기를 만들어보자**
- Viagra 단어외에 영향을 주는 단어들은?
- 오히려 스팸을 제외해주는 단어는 어떻게 찾을까?
- 한번에 여러 단어들을 고려하는 필터기를 만들어보자

**Feature의 확장**<br><br>
$P(spam | viagra)$ &nbsp;&nbsp;( spam : y &nbsp;&nbsp;&nbsp;&nbsp; viagra : x )<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Downarrow$<br>
$P(spam | viagra, hello, lucky, marketing, \cdots)$
- y는 그대로 1또는 0을 가지지만, x는 $x_1, x_2, x_3 \cdots $로 확장됨
- 변수가 많을 때, 조건부 확률의 변화

**[ Multivariate multiplication rule ]**
> $P(Y | X_1 \cap X_2) = \frac{P(Y \cap X_1 \cap X_2)}{P(X_1 \cap X_2)}$<br>
> $P(Y \cap X_1 \cap X_2) = P(Y | X_1, X_2) P(X_1 \cap X_2)$ <br><br>
> $P(X_1, X_2, X_3, \cdots, X_n)$<br>
> $= P(X_1)P(X_2|X_1)P(X_3|X_1, X_2) \cdots P(X_n|X_1 \cdots X_{x-1})$ 

**Problems!**
- 변수가 늘어날수록 급격히 계산이 어려워짐
- Feature의 차원이 증가하면 Sparse Vector가 생성 $\rightarrow$ 확률이 0이 되는 값이 늘어남

#### **[ Naive Bayes Classifier ]**
- 접근방향 : 복잡하게 하지만고 단순(naive)하게 해결하자
- 각 변수의 관계가 독립임을 가정
- 계산이 용이해지고, 성능이 생각보다 좋음..!

**Joint Probability**
> $P(A \cap B) = P(A)P(B)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; if A and B are independent<br><br>
> $P(Y|X_1 \cap X_2) = \frac{P(Y)P(X_1 \cap X_2 | Y)}{P(X_1 \cap X_2)}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$= \frac{P(Y)P(X_1|Y)P(X_2|Y)}{P(X_1)P(X_2)}$
- 이전의 Multivariate multiplication rule보다는 더 간결해짐

**Naive Bayes Classifier**
> ##### $P(Y|X_1 \cap X_2) = \frac{P(Y) P(X_1 \cap X_2 | Y)}{P(X_1 \cap X_2)} = \frac{P(Y)P(X_1|Y)P(X_2|Y)}{P(X_1)P(X_2)}$<br><br>
> ##### <span style='background-color:#fff5b1'>$P(Y_c | X_1, \cdots,X_n) > = \frac{P(Y_c) \prod_{i=1}^n P(X_i |Y_c)}{\prod_{i=1}^n P(X_i)}$</span> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;$Y_c$ : label ($Y_c$ = 0 or 1)
- $X_1$부터 $X_n$까지의 Data가 주어졌을 때, P(Y=1)과 P(Y=0)중에 더 큰 값을 채택하여 모든 데이터마다 진행

**Issue!**
- 너무 많은 확률값$(0 \le P \le 1) \rightarrow$ 0에 수렴하게 되는문제
- 곱하지 말고 더하자 $\rightarrow$ log
<br><br>
> $log({P(Y_c)\prod_{i=1}^n P(X_i|Y_c)}) = log P(Y_c) + \sum_{i=1}^n log P(X_i |Y_c)$
> ##### <span style='background-color:#fff5b1'>$P(Y_c | X_1, \cdots, X_n) = \frac{P(Y_c) \prod_{i=1}^n P(X_i|Y_c)}{\prod_{i=1}^n P(X_i)}$</span> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $Y_c$ : label ($Y_c$ = 0 or 1)
- 분모에서 $X_i$는 피쳐이기 때문에 prod의 결과값은 고정되어있어서 중요하지 않다.
<br><br>
- 확률이 0인 변수들이 존재함 $\rightarrow$ 전체값 0
- 작게나마 확률이 나올 수 있도록변경 $\rightarrow$ **스무딩**
> #### $P(X|Y) = \frac{count(X \cap Y) + k}{count(Y) + (k\;*\;|number\;of\;class|)}$
- 분모분자에 k라는 상수값을 임의로 넣어서 값을 좀 더 평탄하게(generalize) 해준다. 다만 count값들에 비해 k가 커버리면 차이가 더 흐려질수도 있으니 주의
- 스무딩 기법을 사용하면 확률이 0으로 산출되지도 않고 generalize의 효과도 볼수있다.
- binary한 분류문제에서는 Y=1 or 0이 될테니까 number of class=2가 된다
> ##### <span style='background-color:#fff5b1'> $log{P(Y_c)\prod_{i=1}^n P(X_i|Y_c)} = log P(Y_c) + \sum_{i=1}^n logP(X_i | Y_c)$</span>
- 최종식. 우변에서 sum항에 스무딩을 먹여줘서 0인 값이 없도록 한다.
<br><br>
- NB의 핵심 : 최종 식 도출 + 스무딩

<br><br>
### **NB Classifier Implementation**

**Dataset - German Credit**
- 대출 사기인가 아닌가를 예측하는 문제
- 데이터를 NB에 맞도록 간단하게 변환
- Binary 데이터들로 이루어진 대출 사기 데이터

**Data Load, Preprocessing**

In [23]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/TeamLab/machine_learning_from_scratch_with_python/master/code/ch11/fraud.csv')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [25]:
del df["ID"]

Y_data = df.pop("Fraud")
Y_data = Y_data.as_matrix()
Y_data

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [27]:
df.head()

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


**Preprocessing**
- History_arrears = x1, History_current = x2 ... 이런식으로 onehot encoding

In [29]:
x_df = pd.get_dummies(df)
x_df.head()

,History_arrears,History_current,History_none,History_paid,CoApplicant_coapplicant,CoApplicant_guarantor,CoApplicant_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


- onehot encoding의 결과로 x1부터 x10까지의 feature들이 나왔음
- 따라서 x의 벡터들이 주어졌을 때, P(y=1)과 P(y=0)중 더 큰걸 선택할수 있도록 classifier를 만들어줄거임

In [30]:
x_data = x_df.as_matrix()
x_data

array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

**Modeling**
##### $P(Y_c | X_1, \cdots, X_n) = \frac{P(Y_c) \prod_{i=1}^n P(X_i|Y_c)}{\prod_{i=1}^n P(X_i)}$<br>
- 위 식을 따르는 Naive Bayes's Classifier를 만든다.
- $x_1, \cdots ,x_n$ 은 $x_1, \cdots ,x_{10}$ 까지
- 분모는 모든 y의 값에 적용되니 무시해도된다.

In [32]:
P_Y_True = sum(Y_data == True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True, P_Y_False

(0.3, 0.7)

- Y의 Index : 사기가 있는것과 없던것의 인덱스 구분
- np.where() : 괄호안의 조건에 성립하는것의 인덱스값을 반환

In [33]:
# 사기 당한 인덱스들
ix_Y_True = np.where(Y_data)
# 사기 안당한 인덱스들
ix_Y_False = np.where(Y_data == False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12], dtype=int64),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19],
        dtype=int64),))

##### $P(X_i | Y_c) = \frac{Count(X_i \cap Y = 1)}{Count(Y=1)} \cdots$ (Y=0일때도)

In [35]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0)) / sum(Y_data == True)
p_x_y_false = (x_data[ix_Y_False].sum(axis=0)) / sum(Y_data == False)

p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))